<h2>Alberto Moro Carrera</h2>

Quitamos valores aleatorios

In [37]:
import pandas as pd
import numpy as np
np.random.seed(10)


data = pd.read_csv('RutasMaritimasOriginal.csv') #Lista con más de un millon de coordenadas de rutas maritimas 

remove_n = 1046000 #Cantidad de puntos a remover (de 1048576 que acepta Pandas )
drop_indices = np.random.choice(data.index, remove_n, replace=False) #Escoge esa cantidad de indices aleatorios entre todos los que hay en la lista
df = data.drop(drop_indices) #Elimina los elementos en esos indices

print(df.count())
df.to_csv('MaritimasLimpias.csv', sep=";") #Lista con más de un millon de coordenadas de rutas maritimas 


;trip_count;prev_port;next_port;lat;lon;frequency    2575
dtype: int64


De los que queden quitamos los que sean demasiado cercanos

In [41]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from tqdm import tqdm



df = pd.read_csv('MaritimasLimpias.csv', usecols=['punto', 'lat', 'long'], sep=';') #Lista con más de un millon de coordenadas de rutas maritimas 

print(df)

# Define the radius in kilometers
radius = 200

# Convert degrees to radians for latitudes and longitudes
df['lat_rad'] = df['lat'].apply(radians)
df['lon_rad'] = df['long'].apply(radians)

# Function to calculate the distance between two coordinates using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return 6371 * c  # 6371 is the radius of the Earth in km

# Iterate over each row and keep only the coordinates that are more than radius km apart
# Iterate over each row and keep only the coordinates that are more than radius km apart
filtered_coordinates = []
for i, row1 in tqdm(df.iterrows(), total=df.shape[0]):
    keep = True
    sub_df = df.iloc[i+1:]  # create sub-dataframe that contains remaining rows
    for _, row2 in sub_df.iterrows():
        dist = haversine(row1['lat_rad'], row1['lon_rad'], row2['lat_rad'], row2['lon_rad'])
        if dist < radius:
            keep = False
            break
    if keep:
        filtered_coordinates.append([row1['punto'], row1['lat'], row1['long']])

# Convert the filtered coordinates back to a pandas dataframe and save to CSV
df_filtered = pd.DataFrame(filtered_coordinates, columns=['id', 'lat', 'long'])
print(df_filtered.count())
df_filtered.to_csv('MaritimasLimpiasSinCercanas.csv', index=False)



      punto        lat        long
0         1  44.363002  -86.634613
1         2  42.316792  -83.064920
2         3  45.891653  -85.478050
3         4  42.987840  -82.415786
4         5  43.348835  -82.241063
...     ...        ...         ...
2570   2571  -0.199445  117.864012
2571   2572   0.058042  118.381449
2572   2573  -3.401437  117.889875
2573   2574   4.814735  119.623702
2574   2575  25.414350  120.003517

[2575 rows x 3 columns]


100%|██████████| 2575/2575 [01:08<00:00, 37.84it/s] 

id      710
lat     710
long    710
dtype: int64
